In [ ]:
import pandas_utils as pu
import pandas as pd
import numpy as np
import metadata_cleanup as cleanup
import rdkit.Chem as Chem
from rdkit_mol_identifiers import split_inchikey, ensure_smiles_column
import pubchempy as pp

In [ ]:
filename = r"C:\git\msn_library\data\iocb_libraries\IOCB_cmps.tsv"

In [ ]:
filename = r"C:\git\msn_library\data\iocb_libraries\targetmol\targetmol_L6000_natural_products_HTS_2175cmps.parquet"

In [ ]:
df = pu.read_dataframe(filename)

In [ ]:
compounds = pp.get_compounds("alpha-Dihydroartemisinin", "name")
for compound in compounds:
  print(compound.inchikey)

In [ ]:
compounds = pp.get_compounds("C[C@@H]1[C@]2([C@]34[C@@](O[C@](C)(OO3)CC[C@]4([C@H](C)CC2)[H])(O[C@H]1O)[H])[H]", "smiles")
for compound in compounds:
  print(compound.inchikey)

In [ ]:
compound_name = ["aspirin"
                    # "α - Dihydroartemisinin",
                ]

compounds_dicts = {}

for compound in compound_name:
  compounds = pp.get_compounds(compound_name, "name")]
  if compounds:
        compounds_dict[smiles] = compounds[0]
        
for smiles, compound in compounds_dict.items():
    print(f"SMILES: {smiles}")

In [ ]:
df[[ID]]

In [ ]:
df[df["ID"] == "T7059"]

In [ ]:
df[df["smiles"]=="C[C@H]1[C@H](O)O[C@@H]2O[C@@]3(C)CC[C@H]4[C@H](C)CC[C@@H]1[C@@]24OO3"]

In [ ]:
df["isomeric_smiles"] = None
df = df.head(6)
df

In [ ]:
pu.save_dataframe(df, r"C:\git\msn_library\data\iocb_libraries\IOCB_cmps_test.tsv")

In [ ]:
df_template = df[["compound_name", "smiles", "cas", "library_id", "plate_id", "well_location", "unique_sample_id"]].head(6)
df_template

In [ ]:
df[["input_name", "compound_name", "smiles", "monoisotopic_mass", "iocb_name", "isomeric_smiles", "unique_sample_id"]]
# df[["compound_name", "smiles", "iocb_name"]]

In [ ]:
df["split_inchikey"] = [split_inchikey(inchikey) for inchikey in df["inchikey"]]

In [ ]:
pu.save_dataframe(df, filename)

In [ ]:
df[["compound_name", "unique_sample_id", "conc"]]

In [ ]:
df["pubchem_cid"].dtype

In [ ]:
df[["compound_name", "pubchem_cid", "input_pubchem_cid", "monoisotopic_mass", "structure_source"]]

In [ ]:
df[df["unique_sample_id"] == "pluskal_mce_1D3_K17_id"]

In [ ]:
pu.save_dataframe(df, filename)

In [ ]:
df = df.iloc[5640:]

In [ ]:
df

In [ ]:
pu.save_dataframe(df, r"C:\git\msn_library\data\library\mce_library_all_cleaned_molreference_subset2.tsv")

In [ ]:
df["id"].unique()

In [ ]:
df[df["unique_sample_id"] == "pluskal_nencka_mce_5000_A3_id"][["monoisotopic_mass", "compound_name", "molecular_species"]]


In [ ]:
df[df["compound_name"] == "polysignine"][["monoisotopic_mass", "compound_name", "molecular_species", "unique_sample_id"]]


In [ ]:
positive_df["quality_explained_intensity"] = positive_df["quality_explained_intensity"].astype(
  "float")
positive_df["num peaks"] = positive_df["num peaks"].astype("int")
positive_df["quality_explained_signals"] = positive_df["quality_explained_signals"].astype("float")
mslevels = positive_df.groupby("mslevel")
fig, ax = plt.subplots()
# Plot each group as a subplot
for name, group in mslevels:
  group['quality_explained_intensity'].hist(ax=ax, label=name)

# Add a title to the figure
ax.set_title('MSLevel Purity')

# Show the figure
plt.show()
# Create a figure with a single subplot
fig, axs = plt.subplots(len(mslevels), 1, figsize=(10, 10))

# Plot the desired column for each group
for i, (name, mslevel) in enumerate(mslevels):
  mslevel['quality_explained_intensity'].hist(ax=axs[i], label=name, bins=20, width=0.02)
  # Add a title to the subplot
  axs[i].set_title(name)

# Add a title to the figure
fig.suptitle('Your Figure Title')

# Show the figure
plt.show()
file_name = "explained_intensity"

plt.figure(figsize=(13, 9))
ax = sns.histplot(data=positive_df, x="quality_explained_intensity", binwidth=0.01, kde=False)
ax.set(xlabel="Explained Intensity")
# save_fig(file_name)

In [ ]:
wells = ["A01", "A12"]

for well in wells:
  letter = well[0]
  number = int(well[1:])
  correct = f"{letter}{number}"
  print(correct)

In [ ]:
file = r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned.tsv"

In [ ]:
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
df[df["plate_id"] == "07P"]

In [ ]:
df = df[df["plate_id"] != "07P"]

In [ ]:
df

In [ ]:
pu.save_dataframe(df, r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv")

In [ ]:
smiles = "C(C1C(C(C(C(O1)O)N)O)O)O"
mol = Chem.MolFromSmiles(smiles)
smarts = Chem.MolToSmarts(mol)
smarts

In [ ]:
file = r"C:\git\msn_library\data\weizmann\WeizMassV2_cleaned.tsv"
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
df.drop_duplicates("inchikey")

In [ ]:
lib_df[lib_df["inchikey"].duplicated(keep=False)][["inchikey", "split_inchikey", "none"]]


In [ ]:
merged = pd.merge(df, lib_df, on="split_inchikey", how="left")
merged = merged.drop_duplicates(["split_inchikey"])
merged

In [ ]:
merged[merged["split_inchikey"].duplicated(keep=False)][["split_inchikey", "inchikey", "compound_name", "monoisotopic_mass"]]

In [ ]:
pu.save_dataframe(merged, r"C:\git\msn_library\data\weizmann\weizmass_new_split_inchikey_complete_cleaned.tsv")

In [ ]:
filtered = merged[["split_inchikey", "inchikey", "smiles", "inchi", "compound_name", "monoisotopic_mass", "id"]]
filtered

In [ ]:
pu.save_dataframe(filtered, r"C:\git\msn_library\data\weizmann\weizmass_new_split_inchikey_complete_cleaned_column_filtered.tsv")

In [ ]:
pu.read_dataframe(r"C:\git\msn_library\data\weizmann\match_NIHids_to_WizTestSet_pos_complete_table.tsv")

In [ ]:
file = r"C:\git\msn_library\data\acquisition_results\all_lib_only_detected.tsv"
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
df_filter = df[["compound_name", "inchikey", "smiles", "inchi"]].drop_duplicates("inchikey")

In [ ]:
df_filter

In [ ]:
pu.save_dataframe(df_filter, r"C:\git\msn_library\data\acquisition_results\structures_spectral_library_19032024.tsv")

In [ ]:
df = pu.read_dataframe(r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv")

In [ ]:
df["ncgc_id"]

In [ ]:
df

In [ ]:
file = r"C:\git\msn_library\data\acquisition_results\20240527_2batch_all_lib_missing.tsv"


In [ ]:
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
sub = df[(df["library"] == "enam") | (df["library"] == "enammol")][["inchikey", "Catalog ID", "SAMPLEIDDISPLAY", "compound_name", "detected", "library", "unique_sample_id"]]
sub

In [ ]:
pu.save_dataframe(sub, r"C:\git\msn_library\data\acquisition_results\20240527_missing_compounds_enam_enammol.tsv")

In [ ]:
df.groupby(["unique_sample_id"]).count()[["inchikey"]]

In [ ]:
df[(df["plate_id"] == 5004) & (df["well_location"] == "C5")]["unique_sample_id"]

In [ ]:
file = r"C:\git\msn_library\data\public_library\NIST23tandemLib_statistics_standardized.csv"

In [ ]:
df = pu.read_dataframe(file)
df

In [ ]:
df["Positive Mode MS2"].sum() + df["Negative Mode MS2"].sum() 

In [ ]:
df["Positive Mode MSn"].sum() + df["Negative Mode MSn"].sum() 

In [ ]:
df["Positive Mode MSn"].sum() + df["Negative Mode MSn"].sum() + df["Positive Mode MS2"].sum() + df["Negative Mode MS2"].sum() 

In [ ]:
file = r"C:\git\msn_library\data\public_library\mzCloud_04.03.2024_for Customer_standardized.tsv"
df = pu.read_dataframe(file)
df

In [ ]:
df["Spectra Positive Count"].sum() + df["Spectra Negative Count"].sum() 

In [ ]:
file = r"C:\git\msn_library\data\iocb_libraries\Veverka_group\enammol_cleaned.tsv"

In [ ]:
df = pu.read_dataframe(file)
df

In [ ]:
filename = r"C:\git\msn_library\data\DIANA\diana_plate1_test_standardized.tsv"

In [ ]:
df = pu.read_dataframe(filename)
df

In [ ]:
# Filter the DataFrame using regex
filtered_df = df[df['well_location'].str.match(r'^[A-Z]3$')]
filtered_df

In [ ]:
pu.save_dataframe(filtered_df, "C:\git\msn_library\data\DIANA\diana_plate1_test_standardized_A3_Z3.tsv")